# 03_wrangle

In [ ]:
#|default_exp utils.wrangle

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import adulib.utils.wrangle as this_module

In [ ]:
#|export
import pandas as pd
from typing import List

In [ ]:
#|hide
show_doc(this_module.df_to_tsv)

## df_to_tsv

```python
df_to_tsv(df: DataFrame, include_index: bool) -> str
```

Converts a DataFrame to a tab-separated string.

Useful for copying to Excel or Google Sheets.

---


In [ ]:
#|export
def df_to_tsv(df: pd.DataFrame, include_index: bool = False) -> str:
    """
    Converts a DataFrame to a tab-separated string.
    Useful for copying to Excel or Google Sheets.
    """
    return df.to_csv(sep='\t', index=include_index)

In [ ]:
df = pd.DataFrame([
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25}
])
df_to_tsv(df)

'name\tage\nAlice\t30\nBob\t25\n'

In [ ]:
#|hide
show_doc(this_module.df_to_clipboard)

## df_to_clipboard

```python
df_to_clipboard(df: DataFrame, include_index: bool)
```

Converts a DataFrame to a tab-separated string and copies it to the clipboard,

so it can be pasted directly into Google Sheets or Excel.

**Arguments:**
- `df` (*pd.DataFrame*): The DataFrame to copy.
- `include_index` (*bool*): Whether to include the index in the output.

---


In [ ]:
#|export
def df_to_clipboard(df: pd.DataFrame, include_index: bool = False):
    """
    Converts a DataFrame to a tab-separated string and copies it to the clipboard,
    so it can be pasted directly into Google Sheets or Excel.
    
    Args:
        df (pd.DataFrame): The DataFrame to copy.
        include_index (bool): Whether to include the index in the output.
    """
    import pyperclip
    pyperclip.copy(df_to_tsv(df, include_index))

In [ ]:
#|hide
show_doc(this_module.flatten_dict)

## flatten_dict

```python
flatten_dict(d, prefix, sep, keep_unflattened: typing.List[str])
```

Flatten a nested dictionary into a single-level dictionary with compound keys.

This function recursively flattens dictionaries and lists. Nested keys are concatenated 
using a separator (default is an underscore). Lists are flattened by appending the index 
to the key. Nested dictionaries and lists within lists are also handled.

**Arguments:**
- `d` (*dict*): The dictionary to flatten.
- `prefix` (*str*): The base key to use for the current level of recursion. 
Used internally during recursion. Defaults to ''.
- `sep` (*str*): Separator to use between concatenated keys. Defaults to '.'.

**Returns:** *dict*: A new flattened dictionary with compound keys.

---


In [ ]:
#|export
def flatten_dict(d, prefix='', sep='.', keep_unflattened: List[str] = None):
    """
    Flatten a nested dictionary into a single-level dictionary with compound keys.

    This function recursively flattens dictionaries and lists. Nested keys are concatenated 
    using a separator (default is an underscore). Lists are flattened by appending the index 
    to the key. Nested dictionaries and lists within lists are also handled.

    Args:
        d (dict): The dictionary to flatten.
        prefix (str, optional): The base key to use for the current level of recursion. 
                                    Used internally during recursion. Defaults to ''.
        sep (str, optional): Separator to use between concatenated keys. Defaults to '.'.

    Returns:
        dict: A new flattened dictionary with compound keys.
    """
    items = []
    keep_unflattened = keep_unflattened or []
    for k, v in d.items():
        new_key = f"{prefix}{sep}{k}" if prefix else k
        if isinstance(v, dict) and new_key not in keep_unflattened:
            items.extend(flatten_dict(v, new_key, sep=sep, keep_unflattened=keep_unflattened).items())
        elif isinstance(v, list) and new_key not in keep_unflattened:
            for i, item in enumerate(v):
                indexed_key = f"{new_key}{sep}{i}"
                if isinstance(item, dict) and indexed_key not in keep_unflattened:
                    items.extend(flatten_dict(item, indexed_key, sep=sep, keep_unflattened=keep_unflattened).items())
                else:
                    items.append((indexed_key, item))
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
data = {
	'key1' : 'val',
	'key2' : [1, 2, 3],
	'key3' : {
		'foo' : 'bar',
		'baz' : [1, 2, 3],
        'qux' : {
            'key4' : 'value',
            'key5' : [4, 5, 6]
        }
	}
}

flatten_dict(data)

{'key1': 'val',
 'key2.0': 1,
 'key2.1': 2,
 'key2.2': 3,
 'key3.foo': 'bar',
 'key3.baz.0': 1,
 'key3.baz.1': 2,
 'key3.baz.2': 3,
 'key3.qux.key4': 'value',
 'key3.qux.key5.0': 4,
 'key3.qux.key5.1': 5,
 'key3.qux.key5.2': 6}

In [ ]:
flatten_dict(data, keep_unflattened=['key3.qux'])

{'key1': 'val',
 'key2.0': 1,
 'key2.1': 2,
 'key2.2': 3,
 'key3.foo': 'bar',
 'key3.baz.0': 1,
 'key3.baz.1': 2,
 'key3.baz.2': 3,
 'key3.qux': {'key4': 'value', 'key5': [4, 5, 6]}}

In [ ]:
#|hide
show_doc(this_module.flatten_records_to_df)

## flatten_records_to_df

```python
flatten_records_to_df(
   records,
   col_prefix,
   sep,
   max_cols,
   keep_unflattened: typing.List[str]
)
```

Flattens a list of (potentially nested) dictionaries into a pandas DataFrame.

Each dictionary in the input list is flattened using compound keys for nested structures.
Lists within dictionaries are expanded with indexed keys. The resulting DataFrame has one row per record.

**Arguments:**
- `records` (*list*): List of dictionaries to flatten.
- `col_prefix` (*str*): Prefix to add to all column names. Defaults to ''.
- `sep` (*str*): Separator to use between concatenated keys. Defaults to '.'.
- `max_cols` (*int*): Maximum allowed number of columns. Raises ValueError if exceeded.

**Returns:** *pd.DataFrame*: DataFrame with flattened records as rows.

---


In [ ]:
#|export
def flatten_records_to_df(records, col_prefix='', sep='.', max_cols=None, keep_unflattened: List[str] = None):
    """
    Flattens a list of (potentially nested) dictionaries into a pandas DataFrame.

    Each dictionary in the input list is flattened using compound keys for nested structures.
    Lists within dictionaries are expanded with indexed keys. The resulting DataFrame has one row per record.

    Args:
        records (list): List of dictionaries to flatten.
        col_prefix (str, optional): Prefix to add to all column names. Defaults to ''.
        sep (str, optional): Separator to use between concatenated keys. Defaults to '.'.
        max_cols (int, optional): Maximum allowed number of columns. Raises ValueError if exceeded.

    Returns:
        pd.DataFrame: DataFrame with flattened records as rows.
    """
    cols = set()
    flattened_records = []
    for record in records:
        flattened = flatten_dict(record, prefix=col_prefix, sep=sep, keep_unflattened=keep_unflattened)
        flattened_records.append(flattened)
        cols.update(flattened.keys())
        if max_cols is not None and len(cols) >= max_cols:
            raise ValueError(f"Maximum number of columns ({max_cols}) exceeded.")
    return pd.DataFrame(flattened_records)

In [ ]:
records = [
    {'name': 'Alice', 'age': 30, 'address': {'city': 'Wonderland', 'zip': '12345'}},
    {'name': 'Bob', 'age': 25, 'address': {'city': 'Builderland', 'zip': '67890'}},
    {'name': 'Charlie', 'age': 35, 'address': {'city': 'Chocolate Factory', 'zip': '54321'}}
]

flatten_records_to_df(records)

,name,age,address.city,address.zip
0,Alice,30,Wonderland,12345
1,Bob,25,Builderland,67890
2,Charlie,35,Chocolate Factory,54321


In [ ]:
flatten_records_to_df(records, keep_unflattened=['address'])

,name,age,address
0,Alice,30,"{'city': 'Wonderland', 'zip': '12345'}"
1,Bob,25,"{'city': 'Builderland', 'zip': '67890'}"
2,Charlie,35,"{'city': 'Chocolate Factory', 'zip': '54321'}"
